# Create New Project


<p style="color:red;">This is a read-only notebook. It may be customized and run, but settings will only be saved in the newly generated projects.</p>

This notebook is used to create a new project based on a template file. If you are unsure of the terminology used here, please see the Glossary on the <a href="https://we1s.ucsb.edu/research/we1s-tools-and-software/key-tools-and-software/" target="_blank">WE1S Key Tools & Software</a> page or, once you have created a project, in your project's `getting_started.md` file.

Note that if you have previously run this notebook, you may want to clear the outputs before proceeding to ensure that your project has the correct configurations.

## General Project Configuration

The following cell sets configurations that will be used throughout your project. The settings you define here will be written to your project's `config/config.py` file. **If you are unsure what to put here, leave the default settings.**

- `PROJECT_ROOT`: The location where your project will be stored. By default, this is your `write` directory, but you can add subfolders such as `projects` or `projects/2020` to the end of the path if you do not wish to store project at the top level of the `write` directory.
- `PORT`: The server port number where you will run project notebooks. If your server does not use the default port number `11111`, change it here.
- `MONGODB_CLIENT`: If you are using MongoDB as a data source or saving data to MongoDB, you can set the client url here. Otherwise, you can leave the default or change it to an empty string.

In [1]:
PROJECT_ROOT      = '/home'
PORT              = '11111'
MONGODB_CLIENT    = 'mongodb://mongo:27017'

## Configure Metadata

When your project is created, it will have a `README.md` file containing metadata about the project. The configuration values you set in the next cell will be displayed in your project's `README.md` file.

- `project_name`: A unique identifier for the project. It must include only lower case letters, numbers, "-", and "\_". It is helpful for `project_name` identifiers to be semi-human readable. For instance, a `project_name` might be "new_york_times_2010-2015". When your project is created, the `project_name` will be prefixed by a time-date stamp representing when the project was created.
- `project_title`: A human-readable title for the project, which should typically follow standard capitalization rules for titles. Formatting should be indicated with <a href="https://en.wikipedia.org/wiki/Markdown#Example" target="_blank">Markdown</a> For example, "_New York Times_ Articles from 2010-2015".
- `contributors`: The name of the creator or contributors to the project. Projects created by single individuals can give the creator's name in single quotes (e.g. `'John Smith'`). Projects created by more than one person should provide a comma-separated list of quoted names in square brackets (e.g. [`'Jane Smith', 'John Smith'`].
- `description`: A prose description of the project. Formatting should be indicated with <a href="https://en.wikipedia.org/wiki/Markdown#Example" target="_blank">Markdown</a>.

In [2]:
# Project Metadata
project_name      = 'test' # Include only lower case letters, numbers, "-", and "_"
project_title     = 'Test'
contributors      = 'Your name' # String or list
description       = 'Description of the project.'


## Select Project Template Source

A new project is a single directory into which modules are copied from a project template. Project templates are `.tar.gz` archives which you can import to your project directory from a local folder or a url. If you are working in a server environment, the template zip file must be uploaded to the server and imported from the upload directory. Enter either a filepath or a url for the archive file for the `template_source` setting.

To use the latest template, configure `template_source='latest'`. This will download the most recent <a href="https://github.com/whatevery1says/we1s-templates/releases" target="_blank">release from GitHub</a>, so you must have an internet connection.

The `.tar.gz` file is no longer needed after it has been extracted, so by default it is removed from your project folder. If for some reason you want to keep the file, you can do so by setting `remove_package=False`. 

In [3]:
template_source  = 'dist/latest/project_template-0.1.9.tar.gz'
remove_package   = True

## Create Project

This cell will create your project folder, extract the project template to that folder, and provide a link to the new project. If you create project by mistake, the final cell in this notebook provides a method of deleting it.

In [4]:
# Python Imports
import json
import os
import re
import requests
import shutil
import zipfile
from datetime import datetime
from IPython.display import display, HTML
from pathlib import Path

# Import the TemplatePackage class
from template_package import TemplatePackage

# Create the project variables
errors = False
now = datetime.today()
created = now.strftime('%Y-%m-%d %H:%M:%S')
project_full_name =  now.strftime('%Y%m%d_%H%M_') + project_name
project_dir = os.path.join(PROJECT_ROOT, project_full_name)
if isinstance(contributors, list):
    contributors = ', '.join(contributors)

# Validate the project name
if re.search('[^a-z0-9-_]', project_name):
    errors = True
    msg = """Your `project_name` must contain only lower case letters, numbers, "-", and "_".
              Please remove invalid characters and re-run this cell."""
    display(HTML('<p style="color:red;">' + msg + '</p>'))

# Create the template package
package = TemplatePackage()
# Load the archive to the project directory
package.load(template_source, project_dir)
# Extract the archive to the project location with option to remove
archive_filename = str(Path(template_source).name)
archive_filepath = os.path.join(project_dir, archive_filename)
package.extract(archive_filepath, project_dir, remove_package=remove_package)

# Localise datapackage resources
with open(os.path.join(project_dir, 'datapackage.json'), 'r') as f:
    doc = json.loads(f.read())
    for i, item in enumerate(doc['resources']):
        doc['resources'][i] = item['path'].replace('project_template', project_dir)
with open(os.path.join(project_dir, 'datapackage.json'), 'w') as f:
        f.write(json.dumps(doc, indent=2))

# Copy the config values into the DEFAULT config file
with open(project_dir + '/config/config.py', 'r') as f:
    config = f.read()
config = re.sub(r'(PROJECT_ROOT\s+= )\'.+\'', r"\1'" + PROJECT_ROOT + "'", config)
config = re.sub(r'(PORT\s+= )\'.+\'', r"\1'" + PORT + "'", config)
config = re.sub(r'(MONGODB_CLIENT\s+= )\'.+\'', r"\1'" + MONGODB_CLIENT + "'", config)
with open(project_dir + '/config/config.py', 'w') as f:
    f.write(config)

# Handle the README file
readme_template = '# README\n\n'
readme_template += '**Project Name:** ' + project_full_name + '\n'
readme_template += '**Project Title:** ' + project_title + '\n'
readme_template += '**Contributors:** ' + contributors + '\n'
readme_template += '**Created:** ' + created + '\n\n'
readme_template += '**Description:** ' + description + '\n'
# Create a basic README file if it does not exist
if not os.path.exists(os.path.join(project_dir, 'README.md')):
    try:
        with open(os.path.join(project_dir, 'README.md'), 'w') as f:
            f.write(readme_template)
    except IOError:
        errors = True
        display(HTML('<p style="color:red;">Could not create a README.md file.</p>'))
# Otherwise, add the template information to the end of the README file
else:
    try:
        with open(os.path.join(project_dir, 'README.md'), 'r') as f:
            text = f.read()
            text = readme_template + '\n\n' + text
    except IOError:
        errors = True
        display(HTML('<p style="color:red;">Could not open the README.md file.</p>'))
    try:
        with open(os.path.join(project_dir, 'README.md'), 'w') as f:
            f.write(text)
    except IOError:
        errors = True
        display(HTML('<p style="color:red;">Could not modify the README.md file.</p>'))

# Display a success message and a link to the project directory
if errors is False:
    url = PROJECT_ROOT.replace('home/jovyan', 'tree')
    url = os.path.join(url, project_full_name)
    display(HTML('<h4>Your project called ' + project_full_name + ' was created successfully.</h4>'))
    display(HTML('<p>Go to your project directory at <a href="' + url + '" target="top">' + url + '</a> (please make a note of the url).</p>'))
else:
    msg = """The project could not be created. Please check your settings and ensure that the template zip file
              is in the expected location."""
    display(HTML('<p style="color:red;">' + msg + '</p>'))

Template archive extracted.


## Delete a Project

In case you mistakenly create a project and you want to delete it, this cell is provided for convenience. Just configure the path to the project directory and set `delete=True` (by default it is false in case the entire notebook is run at once).

In [5]:
# Configure the path to the project folder.
# By default, it is based on the configurations above -- you may need to re-run the first cell.

PROJECT_ROOT = '/home'
project_dir  = os.path.join(PROJECT_ROOT, project_full_name)
delete       = False

if delete:
    import shutil
    from IPython.display import display, HTML
    try:
        shutil.rmtree(project_dir)
        display(HTML('<p style="color:green;">Project deleted.</p>'))
    except IOError:
        display(HTML('<p style="color:red;">Something went wrong. The project could not be deleted.</p>'))